# Subtract background component from FD-FLIM data
*Matt Renshaw* 
*CALM STP*
*The Francis Crick Institute*
*2025-09-19*  

**Background**: FLIM data contains information from a heterologous population of molecules. This could include donor (in a FRET or non-FRET state) and background. 

**Aim**: Using a measured background sample, subtract the background component from the phase image series to improve contrast of donor fluorescence.

## Table of Contents
1. [Setup](#setup)
2. 

## Setup <a id="setup"></a>
Import libraries

Define username and omero host

In [101]:
import sys
sys.path.append("..")  # go up one level so Python can see `utils`

from utils import Toggel_utils as toggel
from utils import omero_utils as omero

In [102]:
import numpy as np
import pandas as pd
from pathlib import Path
import tifffile
import json
import os
from joblib import Parallel, delayed
from scipy.optimize import curve_fit

In [103]:
username = "rensham"
#omero_host = "omero-training.thecrick.org" # optional: defaults to 'omero-prod.camp.thecrick.org'

## Generate background phase images <a id="get-background"></a>
1. Connect to OMERO
2. Get list of background image IDs
3. Get planes for background images

In [105]:
# connect to OMERO
conn = omero.connect(username)

Connected to OMERO server: omero-prod.camp.thecrick.org


In [94]:
#dataset_id = 11462	
dataset_id = 11468

In [106]:
project_id = 4651

In [109]:
list_of_sample_ids = omero.find_image_ids_in_project_kv_dict (conn, project_id, {"Biological Entity":"3T3 cells", "Treatment":"untreated"})
list_of_sample_ids

[351591,
 351597,
 351598,
 351599,
 351590,
 351601,
 351593,
 351544,
 351560,
 351311,
 351312,
 351313,
 351314,
 351315,
 351316,
 351317,
 351318,
 351319,
 351320,
 351321,
 351322,
 351323,
 351324,
 351325,
 351326,
 351327,
 351328,
 351329,
 351586,
 351587,
 351588,
 351589,
 351330,
 351331,
 351332,
 351333,
 351334,
 351595,
 351596,
 351335,
 351336,
 351337,
 351600,
 351338,
 351339,
 351340,
 351341,
 351342,
 351343,
 351344,
 351345,
 351346,
 351592,
 351594]

In [95]:
list_of_sample_ids = omero.find_image_ids_in_dataset_kv_dict (conn, dataset_id, {"Biological Entity":"Purified protein"})
list_of_sample_ids

[346689,
 346690,
 346691,
 346692,
 346693,
 346694,
 346695,
 346696,
 346697,
 346698,
 346699,
 346700,
 346701,
 346702,
 346703,
 346704,
 346705,
 346706,
 346707,
 346708,
 346709,
 346710,
 346711,
 346712,
 346713,
 346714,
 346715,
 346716]

In [96]:
list_of_bkg_ids = omero.find_image_ids_in_dataset_kv_dict (conn, dataset_id, {"Biological Entity":"Buffer"})
list_of_bkg_ids

[346733, 346717, 346765, 346749]

In [110]:
list_of_bkg_ids = omero.find_image_ids_in_project_kv_dict (conn, project_id, {"Biological Entity":"media", "Treatment":"untreated"})
list_of_bkg_ids

[351616,
 351617,
 351618,
 351619,
 351602,
 351603,
 351604,
 351605,
 351606,
 351607,
 351608,
 351609,
 351610,
 351611,
 351612,
 351613,
 351614,
 351615]

In [ ]:
background_planes_array = []
for image_id in list_of_bkg_ids:
    image_object = omero.get_image (conn, image_id)
    md = omero.get_key_value_metadata (image_object)
    phase_image_id = md["phase_series_image_id"]
    n_phases = md["Toggel-Number of Phases"]
    
    phase_object = omero.get_image (conn, phase_image_id)
    
    # get phase series
    pixels = phase_object.getPrimaryPixels()
    zct_list = [[0, 0, phase] for phase in range(int(n_phases))] # define zct_list
    phase_series = omero.get_planes(zct_list, pixels)
    background_planes_array.append(phase_series)

background_series_stack = np.stack(background_planes_array, axis=0)

Image:351616  Name:"D2-Site_5_T000_Z000_20x_GActin_biosensor_500ms_media_2--ID351231" (owner=rensham)
Image:351231  Name:"2025-09-02_13-54-29-sample-coumarin_ref_4-3ns" (owner=rensham)
Image:351617  Name:"D2-Site_6_T000_Z000_20x_GActin_biosensor_500ms_media_2--ID351308" (owner=rensham)
Image:351308  Name:"2025-09-02_13-54-37-sample-coumarin_ref_4-3ns" (owner=rensham)
Image:351618  Name:"D2-Site_7_T000_Z000_20x_GActin_biosensor_500ms_media_2--ID351256" (owner=rensham)
Image:351256  Name:"2025-09-02_13-54-45-sample-coumarin_ref_4-3ns" (owner=rensham)
Image:351619  Name:"D2-Site_8_T000_Z000_20x_GActin_biosensor_500ms_media_2--ID351182" (owner=rensham)
Image:351182  Name:"2025-09-02_13-54-53-sample-coumarin_ref_4-3ns" (owner=rensham)
Image:351602  Name:"D1-Site_0_T000_Z000_20x_GActin_biosensor_500ms_media_1--ID351239" (owner=rensham)
Image:351239  Name:"2025-09-02_13-21-43-sample-coumarin_ref_4-3ns" (owner=rensham)
Image:351603  Name:"D1-Site_1_T000_Z000_20x_GActin_biosensor_500ms_media_1-

In [98]:
n_areas = len(list_of_bkg_ids)
median_background = []
for phase in range(int(n_phases)):
    planes = background_series_stack[:,phase]
    median_proj = np.nanmedian(planes, axis=0)
    median_background.append(median_proj)

median_background_stack = np.stack(median_background, axis=0)

In [99]:
# Define a sine wave function to fit the data
def sine_wave(x, amplitude, phase_shift, offset):
    phase_shift = (phase_shift + 180) % 360
    return amplitude * np.sin(x + phase_shift) + offset

current_ref_id = None

In [100]:
for sample_id in list_of_sample_ids:
    image_object = omero.get_image (conn, sample_id)
    md = omero.get_key_value_metadata (image_object)
    phase_image_id = md["phase_series_image_id"]
    n_phases = md["Toggel-Number of Phases"]
    reference_image_id = md["reference_image_id"]
    
    phase_object = omero.get_image (conn, phase_image_id)
    
    # get phase series
    pixels = phase_object.getPrimaryPixels()
    zct_list = [[0, 0, phase] for phase in range(int(n_phases))] # define zct_list
    phase_series = omero.get_planes(zct_list, pixels)
    
    background_subtracted = []
    for i, plane in enumerate(phase_series):
        subtract_background = np.subtract(plane, median_background_stack[i])
        background_subtracted.append(subtract_background)
    
    phase_stack_bkg_subtracted = np.stack(background_subtracted, axis=0)

    # Fit processed series    
    # calibrate reference
    
    if (reference_image_id != current_ref_id): 
        # check to see if reference has already been used for calibration
        # load reference image object from omero
        ref_img_obj = conn.getObject("Image", reference_image_id)
        height = ref_img_obj.getSizeY()
        width = ref_img_obj.getSizeX()
        
        # read reference metadata
        ref_metadata = omero.get_key_value_metadata (ref_img_obj)
        frequency = float(ref_metadata["modulation_frequency_Hz"])
        ref_lifetime = float(ref_metadata["reference_lifetime_ns"])
        n_phases = int(ref_metadata["n_phases"])       
        
        # calculate expected values for reference lifetime with modulation frequency and number of phases
        omega, phases, phase_delay_radians, modulation_depth = toggel.calculate_expected_values (frequency, ref_lifetime, n_phases)
        #key_value_pairs.append(["omega", str(omega)])
        #key_value_pairs.append(["frequency_Hz", str(frequency)])
        
        # get phase series
        pixels = ref_img_obj.getPrimaryPixels()
        zct_list = [[0, 0, phase] for phase in range(int(n_phases))] # define zct_list
        ref_phase_series = omero.get_planes(zct_list, pixels)
        
        # calibrate reference
        #ref_mean, A_fitted, phi_fitted, offset_fitted, phases, calibration_m, calibration_phi = toggel.calibrate_reference (ref_phase_series, phases, phase_delay_radians, modulation_depth, width, height)
        # Initialise fitting function
        calibration_fun = toggel.FDFLIM_calibration(
            ref_phase_series, 
            phases, 
            phase_delay_radians, 
            modulation_depth,
            sine_wave
            )
        
        # Run in parallel
        print(f"Calibrating reference image: ID---{reference_image_id}")
        calibration_results = Parallel(n_jobs=-1, backend="loky", verbose=5)(
            delayed(calibration_fun.fit_pixel)(x, y) for x, y in np.argwhere(np.min(ref_phase_series, axis=0) > 0)
        )
        
        # Initialize arrays to hold the modulation and phase results
        calibration_m = np.zeros((height, width)) # modulation depth calibration values
        calibration_phi = np.zeros((height, width)) # phase shift calibration values
        A_fitted = np.zeros((height, width))
        phi_fitted = np.zeros((height, width))
        offset_fitted = np.zeros((height, width))
        ref_mean = np.zeros((height, width))
        
        # populate with results
        for x, y, signal_mean, A_fit, phi_fit, offset_fit, cal_m, cal_phi in calibration_results:
            calibration_m[x, y] = cal_m
            calibration_phi[x, y] = cal_phi
            A_fitted[x, y] = A_fit
            phi_fitted[x, y] = phi_fit
            offset_fitted[x,y] = offset_fit
            ref_mean[x, y] = signal_mean
                
        current_ref_id = reference_image_id

    # Initialise fitting function
    fit_function = toggel.FDFLIM_fitter(                    
            phase_stack_bkg_subtracted,
            ref_mean,
            A_fitted,
            phi_fitted,
            offset_fitted,
            calibration_m,
            calibration_phi,
            phases,
            frequency,
            sine_wave
        )
    
    # Mask of valid pixels
    int_min = np.min(phase_stack_bkg_subtracted, axis=0)
    int_max = np.max(phase_stack_bkg_subtracted, axis=0)
    
    valid_pixels = np.argwhere(((int_min >= 0) & (int_max <= 15000))) # exclude pixels where the lowest value in the phase images is negative and max value is close to saturation/saturated
    
    # Run in parallel
    results = Parallel(n_jobs=-1, backend="loky", verbose=5)(
        delayed(fit_function.fit_pixel)(x, y) for x, y in valid_pixels
    )
    
    # output arrays
    g_values_array = np.full((height, width), np.nan)
    s_values_array = np.full((height, width), np.nan)
    phase_tau_array = np.full((height, width), np.nan)
    mod_tau_array = np.full((height, width), np.nan)
    signal_int_array = np.full((height, width), np.nan)
    
    # populate with results
    for x, y, g, s, phase_tau, mod_tau, sig_int in results:
        # lifetime quality filter
        if ((0 <= g <= 1) & (0 <= s <= 1) & (0 < phase_tau <= 24000) & (0 < mod_tau <= 24000)):
            g_values_array[x, y] = g
            s_values_array[x, y] = s
            phase_tau_array[x, y] = phase_tau
            mod_tau_array[x, y] = mod_tau
            signal_int_array[x, y] = sig_int

    key_value_pairs = [[str(key), str(value)] for key, value in md.items() if not (isinstance(value, float) and np.isnan(value))]
    key_value_pairs.append(["background_image_ids", str(list_of_bkg_ids)])
    key_value_pairs.append(["image_processing", "background subtraction"])

    # processed images to OMERO
    sample_name = image_object.getName()
    image_title = f"BKG_Subtract_{sample_name}"
    pixel_size_um = image_object.getPixelSizeX()
    map_ann = image_object.getAnnotation()
    
    # ensure consistency of pixel type
    image_list = [arr.astype(np.float32) for arr in [signal_int_array, mod_tau_array, phase_tau_array, g_values_array, s_values_array]]
        
    # Stack into a 3D array (C, Y, X)
    image_stack = np.stack(image_list, axis=0)
    
    # list of channel names
    channel_names = ["intensity", "lifetime_modulation", "lifetime_phase", "g_values", "s_values"]
    channel_labels = dict(enumerate(channel_names, start=1))
            
    # image description
    desc = f"Frequency domain FLIM data processed from phase series from omero ID: {phase_image_id}."
    
    # upload image to OMERO
    image_obj = omero.create_image(
        conn, image_title, dataset_id, key_value_pairs, image_stack, channel_names=channel_names, 
        description=desc, sizeZ=1, sizeC=len(image_stack), sizeT=1, pixel_size_um = pixel_size_um, 
        sourceImageId=sample_id
    )

Image:346689  Name:"B2-Site_0_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346169" (owner=rensham)
Image:346169  Name:"2025-09-05_16-05-22-sample-20x_purified_proteins" (owner=rensham)
Calibrating reference image: ID---346044


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  75 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 3336 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 61448 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 144392 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 245768 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done 258048 out of 258048 | elapsed:   25.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 8688 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 40944 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 82416 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 133104 tasks      | elapsed:   13.3s
[Parallel(

Created new image: ID = 350412, Name = BKG_Subtract_B2-Site_0_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346169
Image:346690  Name:"B2-Site_1_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346079" (owner=rensham)
Image:346079  Name:"2025-09-05_16-05-30-sample-20x_purified_proteins" (owner=rensham)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 8688 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 40944 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 82416 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 133104 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 193008 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 258048 out of 258048 | elapsed:   25.3s finished


Created new image: ID = 350413, Name = BKG_Subtract_B2-Site_1_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346079
Image:346691  Name:"B2-Site_2_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346174" (owner=rensham)
Image:346174  Name:"2025-09-05_16-05-38-sample-20x_purified_proteins" (owner=rensham)


...

Created new image: ID = 350437, Name = BKG_Subtract_C4-Site_1_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346130
Image:346715  Name:"C4-Site_2_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346149" (owner=rensham)
Image:346149  Name:"2025-09-05_16-07-16-sample-20x_purified_proteins" (owner=rensham)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 8688 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 40944 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 82416 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 133104 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 193008 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done 258048 out of 258048 | elapsed:   36.2s finished


Created new image: ID = 350438, Name = BKG_Subtract_C4-Site_2_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346149
Image:346716  Name:"C4-Site_3_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346145" (owner=rensham)
Image:346145  Name:"2025-09-05_16-07-24-sample-20x_purified_proteins" (owner=rensham)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 8688 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 40944 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 82416 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 133104 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 193008 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done 258048 out of 258048 | elapsed:   37.0s finished


Created new image: ID = 350439, Name = BKG_Subtract_C4-Site_3_T000_Z000_20x_GActin_biosensor_purified_proteins_plusActing_500ms_2--ID346145


In [60]:
sample_id = list_of_sample_ids[0]

image_object = omero.get_image (conn, sample_id)
md = omero.get_key_value_metadata (image_object)
phase_image_id = md["phase_series_image_id"]
n_phases = md["Toggel-Number of Phases"]
reference_image_id = md["reference_image_id"]

phase_object = omero.get_image (conn, phase_image_id)

# get phase series
pixels = phase_object.getPrimaryPixels()
zct_list = [[0, 0, phase] for phase in range(int(n_phases))] # define zct_list
phase_series = omero.get_planes(zct_list, pixels)

background_subtracted = []
for i, plane in enumerate(phase_series):
    subtract_background = np.subtract(plane, median_background_stack[i])
    background_subtracted.append(subtract_background)

phase_stack_bkg_subtracted = np.stack(background_subtracted, axis=0)

Image:346575  Name:"B3-Site_2_T000_Z000_20x_GActin_biosensor_purified_proteins_500ms_2--ID346090" (owner=rensham)
Image:346090  Name:"2025-09-05_15-31-44-sample-20x_purified_proteins" (owner=rensham)


In [61]:
phase_stack_bkg_subtracted.shape

(12, 504, 512)

In [70]:
# Fit processed series    
# calibrate reference
current_ref_id = None
if (reference_image_id != current_ref_id): 
    # check to see if reference has already been used for calibration
    # load reference image object from omero
    ref_img_obj = conn.getObject("Image", reference_image_id)
    height = ref_img_obj.getSizeY()
    width = ref_img_obj.getSizeX()
    
    # read reference metadata
    ref_metadata = omero.get_key_value_metadata (ref_img_obj)
    frequency = float(ref_metadata["modulation_frequency_Hz"])
    ref_lifetime = float(ref_metadata["reference_lifetime_ns"])
    n_phases = int(ref_metadata["n_phases"])       
    
    # calculate expected values for reference lifetime with modulation frequency and number of phases
    omega, phases, phase_delay_radians, modulation_depth = toggel.calculate_expected_values (frequency, ref_lifetime, n_phases)
    #key_value_pairs.append(["omega", str(omega)])
    #key_value_pairs.append(["frequency_Hz", str(frequency)])
    
    # get phase series
    pixels = ref_img_obj.getPrimaryPixels()
    zct_list = [[0, 0, phase] for phase in range(int(n_phases))] # define zct_list
    ref_phase_series = omero.get_planes(zct_list, pixels)
    
    # calibrate reference
    #ref_mean, A_fitted, phi_fitted, offset_fitted, phases, calibration_m, calibration_phi = toggel.calibrate_reference (ref_phase_series, phases, phase_delay_radians, modulation_depth, width, height)
    # Initialise fitting function
    calibration_fun = toggel.FDFLIM_calibration(
        ref_phase_series, 
        phases, 
        phase_delay_radians, 
        modulation_depth,
        sine_wave
        )
    
    # Run in parallel
    print(f"Calibrating reference image: ID---{reference_image_id}")
    calibration_results = Parallel(n_jobs=-1, backend="loky", verbose=5)(
        delayed(calibration_fun.fit_pixel)(x, y) for x, y in np.argwhere(np.min(ref_phase_series, axis=0) > 0)
    )
    
    # Initialize arrays to hold the modulation and phase results
    calibration_m = np.zeros((height, width)) # modulation depth calibration values
    calibration_phi = np.zeros((height, width)) # phase shift calibration values
    A_fitted = np.zeros((height, width))
    phi_fitted = np.zeros((height, width))
    offset_fitted = np.zeros((height, width))
    ref_mean = np.zeros((height, width))
    
    # populate with results
    for x, y, signal_mean, A_fit, phi_fit, offset_fit, cal_m, cal_phi in calibration_results:
        calibration_m[x, y] = cal_m
        calibration_phi[x, y] = cal_phi
        A_fitted[x, y] = A_fit
        phi_fitted[x, y] = phi_fit
        offset_fitted[x,y] = offset_fit
        ref_mean[x, y] = signal_mean
            
    current_ref_id = reference_image_id

Calibrating reference image: ID---346044


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 8688 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 40944 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 82416 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 133104 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 193008 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 258048 out of 258048 | elapsed:   26.1s finished


In [71]:
# Initialise fitting function
fit_function = toggel.FDFLIM_fitter(                    
        phase_stack_bkg_subtracted,
        ref_mean,
        A_fitted,
        phi_fitted,
        offset_fitted,
        calibration_m,
        calibration_phi,
        phases,
        frequency,
        sine_wave
    )

# Mask of valid pixels
int_min = np.min(phase_stack_bkg_subtracted, axis=0)
int_max = np.max(phase_stack_bkg_subtracted, axis=0)

valid_pixels = np.argwhere(((int_min >= 0) & (int_max <= 15000))) # exclude pixels where the lowest value in the phase images is negative and max value is close to saturation/saturated

# Run in parallel
results = Parallel(n_jobs=-1, backend="loky", verbose=5)(
    delayed(fit_function.fit_pixel)(x, y) for x, y in valid_pixels
)

# output arrays
g_values_array = np.full((height, width), np.nan)
s_values_array = np.full((height, width), np.nan)
phase_tau_array = np.full((height, width), np.nan)
mod_tau_array = np.full((height, width), np.nan)
signal_int_array = np.full((height, width), np.nan)

# populate with results
for x, y, g, s, phase_tau, mod_tau, sig_int in results:
    # lifetime quality filter
    if ((0 <= g <= 1) & (0 <= s <= 1) & (0 < phase_tau <= 24000) & (0 < mod_tau <= 24000)):
        g_values_array[x, y] = g
        s_values_array[x, y] = s
        phase_tau_array[x, y] = phase_tau
        mod_tau_array[x, y] = mod_tau
        signal_int_array[x, y] = sig_int

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 8688 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 40944 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 82416 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 133104 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 193008 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done 258048 out of 258048 | elapsed:   31.8s finished


In [84]:
key_value_pairs = [[str(key), str(value)] for key, value in md.items() if not (isinstance(value, float) and np.isnan(value))]
key_value_pairs.append(["background_image_ids", str(list_of_bkg_ids)])
key_value_pairs.append(["image_processing", "background subtraction"])

In [85]:
# processed images to OMERO
sample_name = image_object.getName()
image_title = f"BKG_Subtract_{sample_name}"
pixel_size_um = image_object.getPixelSizeX()
map_ann = image_object.getAnnotation()

# ensure consistency of pixel type
image_list = [arr.astype(np.float32) for arr in [signal_int_array, mod_tau_array, phase_tau_array, g_values_array, s_values_array]]
    
# Stack into a 3D array (C, Y, X)
image_stack = np.stack(image_list, axis=0)

# list of channel names
channel_names = ["intensity", "lifetime_modulation", "lifetime_phase", "g_values", "s_values"]
channel_labels = dict(enumerate(channel_names, start=1))
        
# image description
desc = f"Frequency domain FLIM data processed from phase series from omero ID: {phase_image_id}."

# upload image to OMERO
image_obj = omero.create_image(
    conn, image_title, dataset_id, key_value_pairs, image_stack, channel_names=channel_names, 
    description=desc, sizeZ=1, sizeC=len(image_stack), sizeT=1, pixel_size_um = pixel_size_um, 
    sourceImageId=sample_id
)

Created new image: ID = 348716, Name = BKG_Subtract_B3-Site_2_T000_Z000_20x_GActin_biosensor_purified_proteins_500ms_2--ID346090


In [73]:
key_value_pairs = [[str(key), str(value)] for key, value in pos_dict.items() if not (isinstance(value, float) and np.isnan(value))]
key_value_pairs.append(["MicroManager_version", str(mm_version)])
key_value_pairs.append(["Interval_ms", str(interval_ms)])
key_value_pairs.append(["MicroManager_filename", str(filename)])

'B3-Site_2_T000_Z000_20x_GActin_biosensor_purified_proteins_500ms_2--ID346090'

In [74]:
sample_object_id

NameError: name 'sample_object_id' is not defined

In [20]:
image_object = omero.get_image (conn, 346626)

Image:346626  Name:"C5-Site_1_T000_Z000_20x_GActin_biosensor_purified_proteins_500ms_2--ID346064" (owner=rensham)


In [21]:
md = omero.get_key_value_metadata (image_object)

In [23]:
md["Toggel-Number of Phases"]

'12'

In [16]:
phase_image_id = md["phase_series_image_id"]
phase_image_id

'346064'

In [17]:
phase_object = omero.get_image (conn, phase_image_id)

Image:346064  Name:"2025-09-05_15-34-18-sample-20x_purified_proteins" (owner=rensham)


In [18]:
phase_object.getSizeT()

12

In [ ]:
phase_object = omero.get_image (conn, phase_image_id)
n_phases = phase_object.getSizeT()
# get phase series
pixels = image_object.getPrimaryPixels()
zct_list = [[0, 0, phase] for phase in range(n_phases)] # define zct_list
phase_series = omero.get_planes(zct_list, pixels)